# Howso Engine Intro 

## Overview 

 
Howso Engine is a generalized Machine Learning (ML) and Artificial Intelligence platform that creates powerful decision-making platforms that are fully explainable, auditable, and editable. Howso Engine uses Instance-Based Machine Learning which stores instances, i.e., data points, in memory and makes predictions about new instances given their relationship to existing instances. This technology harnesses a fast spatial query system and techniques from information theory to enhance both performance and accuracy. 

 

Howso Engine is unique in that it utilizes a one-model approach (the data is the model) that enables the user to accomplish: 

 

- "Targetless Learning" - allows the user to predict and characterize any set of variables 

- Supervised Learning 

- Unsupervised Learning 

- Reinforcement Learning 

- Online Learning 

- Discriminative and Generative Modeling 

- Model Interpretation and Explanation 

- Data Imputation 

 

In this notebook we will explore the most basic workflow for training Howso Engine and making predictions. This will enable us to become familiar with basic Howso Engine terminologies, including: 

 

- Action Features 

- Context Features 

- Cases 

- Trainee 

- Train 

- Analyze 

- React 

## Recipe Goals:

This notebook will provide a demonstration of a basic Howso Engine workflow.

In [1]:
from pathlib import Path

import pandas as pd
from pmlb import fetch_data

from howso.engine import (
    load_trainee,
    Trainee,
)
from howso.utilities import infer_feature_attributes

# Step 1: Import Data

Our example dataset for this recipe is the well known `Adult` dataset. This dataset consists of 14 Context Features and 1 Action Feature. The Action Feature in this version of the `Adult` dataset has been renamed to `target` and it takes the form of a binary indicator for whether a person in the data makes more than $50,000/year (*target*=1) or less (*target*=0).

### **Definitions:**

**`Action Feature`:** A feature that the user would like to discriminatively predict or generatively synthesize. In traditional ML these may be referred to as targets, target features, or dependent variables.

**`Context Feature`:** A feature that is used as an input to the model that creates a context from which a prediction is derived. In traditional ML these are often referred to as features, inputs, predictors, or independent features.

In [9]:
df = pd.read_csv("../data/adult.data")
display(df)

,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32555,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32556,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32557,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32558,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [27]:
df = fetch_data('adult', local_cache_dir="../data/adult")

# Subsample the data to ensure the example runs quickly
df = df.sample(2000)

# Split out the last row for a prediction set and drop the Action Feature
test_case = df.iloc[[-1]].copy()

df.drop(df.index[-1], inplace=True)

test_case = test_case.drop('target', axis=1)

display(df)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
18154,29.0,4,439263.0,11,9.0,2,11,0,4,1,0.0,0.0,35.0,29,1
43566,29.0,4,153475.0,1,7.0,0,7,4,4,0,0.0,0.0,40.0,39,1
34386,33.0,4,202339.0,1,7.0,4,12,4,4,0,0.0,0.0,34.0,39,1
13784,73.0,0,191394.0,14,15.0,2,0,0,4,1,0.0,0.0,40.0,39,1
950,44.0,7,174675.0,11,9.0,2,10,5,4,0,0.0,0.0,40.0,39,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29920,36.0,2,103886.0,15,10.0,0,6,1,4,1,0.0,0.0,50.0,39,1
37123,35.0,6,346605.0,15,10.0,0,8,4,4,0,0.0,0.0,20.0,39,1
46252,31.0,4,143851.0,7,12.0,2,10,5,4,0,0.0,0.0,40.0,39,0
1705,35.0,4,168817.0,11,9.0,4,7,1,4,1,0.0,0.0,40.0,39,1


In [25]:
df = fetch_data('adult', local_cache_dir="../data/adult")

# Subsample the data to ensure the example runs quickly
df = df.sample(2000)

# Split out the last row for a prediction set and drop the Action Feature
test_case = df.iloc[[-1]].copy()
df.drop(df.index[-1], inplace=True)
test_case = test_case.drop('target', axis=1)

# Step 2: Feature Mapping

Typically, an exploratory analysis is done on the data to get a general feel of the descriptive statistics and data attributes. 

Methods like `describe` from a Pandas dataframe often automatically present these types of information of interest to a user, as shown below. While informative, these descriptive statistics are often used as a sanity check pre- and post-modeling and a model typically doesn't actually use any of these feature attributes.

In [26]:
df.describe()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
count,1999.000000,1999.000000,1.999000e+03,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000
mean,38.663332,3.853427,1.905301e+05,10.322661,10.110055,2.612806,6.677839,1.446723,3.681341,0.667334,1142.949475,106.974987,40.499750,36.980990,0.755878
std,13.672423,1.439131,1.103488e+05,3.922988,2.561338,1.520022,4.198558,1.596743,0.834688,0.471286,7259.850468,456.369197,12.015375,7.208648,0.429673
min,17.000000,0.000000,1.967800e+04,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000
25%,28.000000,4.000000,1.165470e+05,9.000000,9.000000,2.000000,3.000000,0.000000,4.000000,0.000000,0.000000,0.000000,40.000000,39.000000,1.000000
50%,37.000000,4.000000,1.778580e+05,11.000000,10.000000,2.000000,7.000000,1.000000,4.000000,1.000000,0.000000,0.000000,40.000000,39.000000,1.000000
75%,48.000000,4.000000,2.366135e+05,12.000000,13.000000,4.000000,10.000000,3.000000,4.000000,1.000000,0.000000,0.000000,45.000000,39.000000,1.000000
max,90.000000,7.000000,1.366120e+06,15.000000,16.000000,6.000000,14.000000,5.000000,4.000000,1.000000,99999.000000,4356.000000,99.000000,40.000000,1.000000


In the Howso Engine workflow, feature attributes are an essential part of model building and usage. By incorporating certain feature attributes into the model itself, Howso Engine gains another layer of information that will help in fine-tuning the results. 

In order to assist the user with defining the feature attributes, Howso provides the `infer_feature_attributes` function that automatically processes the dataset for the user.

These feature attributes are based on the existing data, rather than exact descriptive statistics. This is why, for example, the min and max bounds on continuous features are not the exact min and max values of the dataset, but rather an expanded version of those min and max values to allow for some variation. However, users are encouraged to manually adjust feature attributes such as bounds if the properties of any features in the data are well defined.

Additionally, Howso always encourages users to inspect the feature attributes of their data. Incorrect feature attributes can severely impact the accuracy of the model. Specific errors to lookout for are incorrect values for 'type' or 'bounds'.

In [4]:
# Infer features attributes
features = infer_feature_attributes(df)
features.to_dataframe()

type decimal_places  bounds                        \
                                              min        max allow_null   
age             continuous              0     7.0      148.0       True   
workclass          nominal              0     NaN        NaN      False   
fnlwgt          continuous              0  8103.0  1202604.0       True   
education          nominal              0     NaN        NaN      False   
education-num   continuous              0     1.0       20.0       True   
marital-status     nominal              0     NaN        NaN      False   
occupation         nominal              0     NaN        NaN      False   
relationship       nominal              0     NaN        NaN      False   
race               nominal              0     NaN        NaN      False   
sex                nominal              0     NaN        NaN      False   
capital-gain    continuous              0     0.0   162755.0       True   
capital-loss    continuous              0     0.0     2981.0       True   
hours-per-week  continuous              0     1.0      148.0       True   
native-country     nominal              0     NaN        NaN      False   
target             nominal              0     NaN        NaN      False   

               data_type original_type       
                             data_type size  
age               number       numeric    8  
workclass         number       integer    8  
fnlwgt            number       numeric    8  
education         number       integer    8  
education-num     number       numeric    8  
marital-status    number       integer    8  
occupation        number       integer    8  
relationship      number       integer    8  
race              number       integer    8  
sex               number       integer    8  
capital-gain      number       numeric    8  
capital-loss      number       numeric    8  
hours-per-week    number       numeric    8  
native-country    number       integer    8  
target            number       integer    8

To prepare to measure model performance, we also define our action features and context features that we want to use in our experiment.

In [5]:
# Specify Context and Action Features
action_features = ['target']
context_features = features.get_names(without=action_features)

> **Note:** Train Test Split

To gauge model performance, train-test splits are often used in traditional machine learning workflows. Howso Engine does not require the use of train-test split for validation. Please see recipe `6-validation.ipynb` for further explanation. Therefore, we will not use train-test splits our recipes unless the test set serves a specific purpose.

# Step 3: Create Trainee

To begin the Howso Engine workflow, a Trainee is created to act as a base for all of our ML needs. In all subsequent notebooks, we will refer to Howso Engine's model as the Trainee.

### **Definitions:**

**`Trainee`:** A collection of Cases that comprise knowledge. It also includes metadata and parameters. In traditional ML this is referred to as a model.

**`Case`:** A set of feature values representing an observation. In traditional ML, a Case is sometimes referred to as an "observation", "record", or "data point". In database terms, a Case would be a row of values. For supervised learning a Case is a set of Context values and Action values. For unsupervised learning a Case is just a set of feature values. 

In [6]:
# Create the Trainee
t = Trainee(features=features)

# Step 4: Preprocessing and Training

One benefit of Howso Engine is that most standard forms of data pre-processing such as one-hot encoding and standardization are **NOT** needed, which is in contrast to many modern ML models. This does not include more sophisticated forms of pre-processing such as feature selection or feature engineering, which may still be useful. Fitting is also done in two steps in Howso Engine.

### **Definitions:**

**`Train`:** To add a case or set of cases to the Trainee, making the information contained in these cases available for downstream tasks.

**`Analyze`:** Tune internal parameters to improve performance and accuracy of predictions and metrics. Analysis may be targeted or targetless.  Targetless analysis provides the best balanced set of parameters if an Action Feature is not specified, along with a performance boost while targeted analysis provides a boost to accuracy towards the specified action features. 


In [7]:
# Train
t.train(df)

# Analyze the Trainee
# (By specifying action_features, this becomes a Targeted analysis)
t.analyze(context_features=context_features, action_features=action_features)

# Step 5: Results

Once Howso Engine is trained and analyzed, it provides the user with a variety of ML capabilities. At this stage in the Howso Engine workflow, a typical use case would be to evaluate the accuracy of the Trainee, which is performed by the `react` method. This is equivalent to `predict` in many traditional Machine Learning workflows, although the `react` method is not solely used for supervised predictions as detailed in subsequent recipes. 

Since we are not using a train-test split, we will use the `react_aggregate` method, which performs a `react` on each of the cases that is trained into the model (or a sample for very large models), then computes the desired metrics. Alternatively, a standard `react` call may be used on untrained data for prediction.

The accuracy is calculated internally as shown in the code below and this is the recommended accuracy metric for nominal action features. Further explanations are available in recipe `6-validation`.

### **Definitions:**

**`React`:** Exposing a Trainee to a new Case's Context Features and an Action Feature for that case is returned. In traditional ML this is often referred to predicting or labeling.

In [8]:
# Recommended metrics
stats = t.react_aggregate(
    action_feature=action_features[0],
    details = {
        "prediction_stats": True,
        "selected_prediction_stats": ["all"]
    }
)


In [9]:
# The confusion matrix can be retrieved
print("Howso Prediction Results - Confusion Matrix")
matrix = pd.DataFrame(stats["target"]["confusion_matrix"]["matrix"])
matrix.index.name = "Predicted"
matrix.columns.name = "Actual"
display(matrix)

Howso Prediction Results - Confusion Matrix


Actual,1,0
Predicted,,
1,678,112
0,78,132


### Sparse Confusion Matrices

By default, Howso confusion matrices are returned in sparse format. This means that for nominal features, only classes that have counts above a certain threshold are returned individually as columns and rows in the confusion matrices. For all classes who do not meet this threshold, their values are lumped together into a ``leftover`` category, where ``leftover_correct`` and ``leftover_incorrect`` represents the results for these lumped classes. 

To control the class count threshold for sparsifying the confusion matrix, please use the ``confusion_matrix_min_count`` parameter in``react_aggregate``.

In [10]:
print(f"Leftover correct: {stats['education']['confusion_matrix']['leftover_correct']}")
print(f"Leftover incorrect: {stats['education']['confusion_matrix']['leftover_incorrect']}")

Leftover correct: 12
Leftover incorrect: 188


We can see that the classes present in the confusion matrix all have counts that are greater than the ``confusion_matrix_min_count`` threshold, which is 10 by default.

In [11]:
# Sparse Confusion Matrix
sparse = pd.DataFrame(stats["education"]["confusion_matrix"]["matrix"]).fillna(0)
sparse

,11,9,7,15
11,232,40,16.0,121
9,16,87,0.0,17
15,58,29,0.0,71


Another way to use the `react_aggregate` method is to utilize the `condition` parameter. Specifying a condition allows the user to get prediction stats on the subset of cases that matches the specified condition.

For example, we can use the `condition` parameter to find the accuracy, precision, and recall on the subsets of the dataset with each value for Sex.

In [12]:
# Condition takes a dictionary of feature name to feature value for nominal features
sex_1_stats = t.react_aggregate(
    action_feature=action_features[0],
    details = {
        "prediction_stats": True,
        "action_condition": {'sex': 1}
    }
)['target'].round(2)
print(f'Howso Prediction Results On Cases with Sex=1 - Accuracy: {sex_1_stats["accuracy"]}, Precision: {sex_1_stats["recall"]}, and Recall: {sex_1_stats["precision"]}')

sex_0_stats = t.react_aggregate(
    action_feature=action_features[0],
    details = {
        "prediction_stats": True,
        "action_condition": {'sex': 0}
    }
)['target'].round(2)
print(f'Howso Prediction Results On Cases with Sex=0 - Accuracy: {sex_0_stats["accuracy"]}, Precision: {sex_0_stats["recall"]}, and Recall: {sex_0_stats["precision"]}')

Howso Prediction Results On Cases with Sex=1 - Accuracy: 0.79, Precision: 0.75, and Recall: 0.77
Howso Prediction Results On Cases with Sex=0 - Accuracy: 0.9, Precision: 0.66, and Recall: 0.71


This type of analysis can help users understand the regions of their data that are less predictable, which may provide insight on how data collection can be improved.

# Step 6: Inference

Once Howso Engine is trained and analyzed, it provides the user with a variety of ML capabilities. One of the core capabilities in engine is prediction, which Engine infers a action value based on the given context values. This prediction is a standard output when using `react` and the prediction can be found by navigating the output as shown.

In [13]:
results = t.react(
    test_case,
    context_features=context_features,
    action_features=action_features,
)

predictions = results['action'][action_features]
predictions

,target
0,1


### Predict Function

A `predict` method is also provided as an ease-of-use function that provides the same results as the method shown above using `react` and directly returns the predictions without requiring the user to navigate the output.

In [14]:
predictions = t.predict(
    test_case,
    context_features=context_features,
    action_features=action_features,
)

predictions

,target
0,1


# Step 7: Saving, Loading, and Deleting from your local disk

These methods work for `HowsoDirectClient` and its subclasses.

### Saving and Loading

When saving to disk, a filepath or filename may provided. The filepath provided may be a relative or absolute path. The filepath can include the desired filename with a `.caml` extension. If no filename is at the end of the filepath, the Trainee ID will be used. If just the filename is provided, then the current working directory will be used as the filepath. 

> **Note:** It is recommended, however not necessary that the filename match the trainee ID


### Deleting

When deleting a Trainee, the `delete` method deletes the trainee from the last saved or loaded disk location, as well as memory. If the Trainee has not been saved, `delete` can also be used to just remove a Trainee from memory.

When Trainees are saved, a `.caml` file and a `.txt` version file is saved. The `delete` method assumes that the prefix to the version filename is the same as the `.caml` trainee filename.

In [15]:
from howso.direct import HowsoDirectClient

# Clients that extend `HowsoDirectClient` will have local file operations
if isinstance(t.client, HowsoDirectClient):
    # Example filepath (same as default)
    cwd = Path.cwd()
    file_path=f"{cwd}/engine_intro_trainee.caml"

    # Saving
    t.save(file_path=file_path)

    # Loading
    t = load_trainee(file_path=file_path)

    # Deleting
    t.delete()

# Conclusion:

Howso Engine provide users with powerful tools to perform a variety of data analysis and prediction tasks. However, making predictions may not suffice if these tools are used for decision-making. As an example with this specific `Adult` dataset, if the user were a bank predicting whether an individual will make over or under $50k in the future for loan decisions, the outcome of prediction efforts might adversely affect the livelihood of that individual as well as the bank if the prediction is inaccurate or biased. Ideally, a predictive tool should provide both accurate results and interpretability so that it can be held accountable for its predictions. Unfortunately, traditional ML techniques often leave a lot to be desired in terms of interpretability. Because of its instance-based ML nature, Howso Engine is inherently interpretable.